# Android IV

## Sensors
- The hardware on Android devices is usually equipped with various sensors that can also cause events
    - A list of Sensors can be found at  https://source.android.com/devices/sensors/sensor-types
- These event must be attached programatically
- The events are registed with a SensorManager object by passing the listener object and the sensor to listen for events on
```java
SensorManager is provided by the OS and retrieved using
mySensorManager = (SensorManager) getSystemService(
    Context.SENSOR_SERVICE);
```

## SensorEventListener
- Just like for input events, the sensor events are handled by classes that implement SensorEventListener
- This provides two methods that need to be overridden
    - `onSensorChanged(SensorEvent event)` which is called when the sensor value changes
    - `onAccuracyChanged(SensorEvent event, int accuracy)` which is called when the accuracy of the sensor changes for some reason

## Biding a Listener to a Sensor
- To attach the listener, an object representing the sensor must be retrieved from the sensor manager object
```java
temp = mSensorManager.getDefaultSensor(Sensor.TYPE_AMBIENT_TEMPERATURE);
```
- The listener is then attached using registerListener and passing the listener, the sensor, and the delay between sensor events
```java
mySensorManager.registerListener(new SensorEventListener(), temperature, SensorManager.SENSOR_DELAY_NORMAL);
```

## Getting Sensor Values
- The `SensorEvent` object passed in has a field `values` which is an array of floats
    - The readings of the sensor are held here
- To determine which index holds which value, consult the list at https://developer.android.com/reference/android/hardware/SensorEvent.html#values
- For example the Gyroscope has 3 values
    - `values[0]` - speed around x axis
    - `values[1]` - speed around y axis
    - `values[2]` - speed around z axis

## Using Location 
- When building an app there are two different APIs that can be used for location
    - The `android.location` package
        - Provides low level access to location API
        - Doesn't require Google Play Services, is soley part of the Android system
    - The `com.google.android.gms.location` package
        - Higer level API
        - Requires Google approved device
        - Can be updated seperately from the OS

## Play Services API Location
- To use location from play services, first request permission for location
```xml
<uses-permission android:name="android.permission.ACCESS_COARSE_LOCATION"/>
```
- Using the Play Services API requires Java classes that are not normally part of an Android Studio project
    - To add them, add the following line to the `build.gradle (Module: app)` file under dependencies
```
implementation 'com.google.android.gms:play-services-location:11.6.0'
```

## Play Services API Location
- Then a new client is created with specifications for the required API and the behavior on connection to that API
```java
mGoogleApiClient = LocationServices.getFusedLocationProviderClient()
```
- Use `getLastLocation` method to attach listeners to a task
```
mGoogleApiClient.getLastLocation().addOnSuccessListener(
       new OnSuccessListener<Location>() {
           @Override
           public void onSuccess(Location location) {
           ...
```

## Geocoding
- The process of turning a longitude and latitude into an address is known as reverse geocoding
- The `Geocoder` class has method to perform both forward and reverse geocoding
    - `getFromLocation`
    - `getFromLocationName`

## Getting Location Updates
- To get frequent updates on the location, call the `requestLocationUpdates` method of the API
- This method requires two parameters, a `LocationRequest` object, and a `LocationCallback` object
- `LocationRequest` is an object used to provide settings about the request, ie how frequent you should recieve updates, the priority of the updates, etc.
` LocationCallBack` is an object that overrides the default `onLocationResult` method, which is fired when the location is updated

## Services
- So far we have been focusing on one major Android component, activities
- Not everything needs a UI
    - In these cases, a service is a good alternative
- Still uses `Intent`s to start
- Implementation should extend `Service`
- Service inherits fron `Context`

## Services in the Manifest
- Services need to be declared in the Manifest
```xml
<service android:name="MyService" 
         exported="False|True"
         />
```
- Setting `exported` to false prevent other apps from using your service

## Started Services
- Started using `startService`
- Once started, can run forever
    - Can run after component that started it is destroyed
- Not designed to send information back to the starting component
    - By default, starts in same thread as launching activity/service
- May not be supported in the future

## The Service Class (for Started Services)
- In addition to extending the standard class `Service`, implement the following methods
    - `onCreate`
    - `onStartCommand`
- To stop the service
    - call `stopSelf()` from with in `Service` class
    - call `stopService(Intent)` from another component

## Common Uses of Services
- Downloading a large file
- Commits something to a database
- Plays music

## Bound Services
- A bound service only exists as long as a component is bound to it
    - Multiple components can bind to a service at one time
- Bound services are meant to provide an interaction back and forth with the calling component
- Started with `bindService(Intent,ServiceConnection,flags)`
- Listen to the `onServiceConnected` of the `ServiceConnection`
    - This gets an `IBinder` object that can be used to interact with the service
    - The `IBinder` is returned from the service from the method `onBind`

## Broadcasts
- If we just want to send a message to a wide variety of activities or apps, we don't need a service
    - We can use broadcast intents
- Sent using `sendBroadcast()`
- Recieved using a subclass of `BroadcastReciever`
    - Must be registered in Manifest.xml

## Sending Broadcasts
- A broadcast sends an intent, which should be created by specifying an action
    - The action can be a custom action, if so, it should be a class constant string
- Any app on your device can possibly listen to this broadcast
    - An optional permission can be suppied as part of the intent to prevent this
- Not simple to send new broadcast types between apps

## Recieving Broadcasts
- Create a new class in your app that extends `BroadcastReciever`
    - Override the `onRecieve` method
- Register your reciever using the `<reciever>` tag
    - Use intent filters to only process certain broadcasts
    - Android has slowly been limiting the Broadcast you can listen to